In [1]:
from transformers import BertConfig, BertTokenizer, BertTokenizerFast, TFBertForMaskedLM, AdamWeightDecay, DataCollatorForLanguageModeling
import tensorflow as tf
from datasets import load_dataset
from collections import Counter
import os.path

In [2]:
# Load dataset

dataset_path = "/Users/Gianni/semiolog/models/en_bnc_berttest/corpus/"
dataset_files = ["train","dev","test"]
extension = "text"
dataset = load_dataset(extension, data_files={f:dataset_path+f+".txt" for f in dataset_files}) #, split=["train[:1%]", "dev[:1%]"])


Using custom data configuration default-4944b41e8dad232a
Reusing dataset text (/Users/Gianni/.cache/huggingface/datasets/text/default-4944b41e8dad232a/0.0.0/e16f44aa1b321ece1f87b07977cc5d70be93d69b20486d6dacd62e12cf25c9a5)
100%|██████████| 3/3 [00:00<00:00, 423.80it/s]


In [3]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 970209
    })
    dev: Dataset({
        features: ['text'],
        num_rows: 4851
    })
    test: Dataset({
        features: ['text'],
        num_rows: 4852
    })
})

In [4]:
# Build vocabulary of segmented sentences and save it to a vocab.txt file

if not os.path.isfile("vocab.txt") or True:
    tokens = []

    for sent in dataset["train"]["text"]:
            tokens.extend(sent.split())
    tokens_count = Counter(tokens) # This could probably be done directly on the counter
    vocab = [token for token, freq in tokens_count.most_common()]

    with open("vocab.txt", 'w') as f:
        for token in ["[PAD]", "[SEP]", "[CLS]", "[MASK]", "[UNK]"] + vocab:
            f.write("%s\n" % token)

In [5]:
tokenizer= BertTokenizer(
        vocab_file = "vocab.txt",
        do_lower_case=True,
        do_basic_tokenize=True,
        never_split=None,
        unk_token="[UNK]",
        sep_token="[SEP]",
        pad_token="[PAD]",
        cls_token="[CLS]",
        mask_token="[MASK]",
        tokenize_chinese_chars=True,
        strip_accents=None,
)

tokenizer_fast = BertTokenizerFast(
        vocab_file="vocab.txt",
        tokenizer_file=None,
        do_lower_case=True,
        unk_token="[UNK]",
        sep_token="[SEP]",
        pad_token="[PAD]",
        cls_token="[CLS]",
        mask_token="[MASK]",
        tokenize_chinese_chars=True,
        strip_accents=None,
)

In [6]:
def tokenize_function(examples):
    return tokenizer(examples["text"])

In [8]:
tokenized_datasets = dataset.map(tokenize_function, batched=True, batch_size=10000, num_proc=8, remove_columns=["text"])
# tokenized_datasets_dev = dataset["dev"].map(tokenize_function, batched=True, batch_size=10000, num_proc=8, remove_columns=["text"])

 #0:   0%|          | 0/13 [00:00<?, ?ba/s]
























 #0:   8%|▊         | 1/13 [00:04<00:50,  4.18s/ba]
















 #0:  15%|█▌        | 2/13 [00:08<00:49,  4.47s/ba]


































 #0:  23%|██▎       | 3/13 [00:13<00:47,  4.74s/ba]




















 #0:  31%|███       | 4/13 [00:18<00:41,  4.65s/ba]

















 #0:  38%|███▊      | 5/13 [00:22<00:36,  4.61s/ba]




















 #0:  46%|████▌     | 6/13 [00:27<00:31,  4.55s/ba]




















 #0:  54%|█████▍    | 7/13 [00:31<00:27,  4.55s/ba]















 #0:  62%|██████▏   | 8/13 [00:36<00:22,  4.54s/ba]




















 #0:  69%|██████▉   | 9/13 [00:40<00:17,  4.49s/ba]




























 #0:  77%|███████▋  | 10/13 [00:45<00:13,  4.62s/ba]




















 #0:  85%|████████▍ | 11/13 [00:50<00:09,  4.69s/ba]














 #5: 100%|██████████| 13/13 [00:54<00:00,  4.17s/ba]







 #0:  92%|█████████▏| 12/13 [00:54<00:04,  4.52s/ba]




 #3: 100%|███████

In [10]:
tokenizer.decode(tokenized_datasets["train"][0]["input_ids"])

'[CLS] itwould control referrals to provid ers outsidethe district and itwould pay forthem at negoti ated prices [SEP]'

In [ ]:
model.fit(train_set, validation_data=validation_set, epochs=2)

2021-12-08 00:32:49.622711: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


Epoch 1/2


2021-12-08 00:32:55.524026: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


24/75 [========>.....................] - ETA: 3:48 - loss: 9.6150

KeyboardInterrupt: 

In [ ]:
model.fit(train_set, validation_data=validation_set, epochs=2)

2021-12-08 00:32:49.622711: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


Epoch 1/2


2021-12-08 00:32:55.524026: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


24/75 [========>.....................] - ETA: 3:48 - loss: 9.6150

KeyboardInterrupt: 

In [ ]:
model.fit(train_set, validation_data=validation_set, epochs=2)

2021-12-08 00:32:49.622711: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


Epoch 1/2


2021-12-08 00:32:55.524026: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


24/75 [========>.....................] - ETA: 3:48 - loss: 9.6150

KeyboardInterrupt: 

In [ ]:
model.fit(train_set, validation_data=validation_set, epochs=2)

2021-12-08 00:32:49.622711: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


Epoch 1/2


2021-12-08 00:32:55.524026: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


24/75 [========>.....................] - ETA: 3:48 - loss: 9.6150

KeyboardInterrupt: 

In [ ]:
model.fit(train_set, validation_data=validation_set, epochs=2)

2021-12-08 00:32:49.622711: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


Epoch 1/2


2021-12-08 00:32:55.524026: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


24/75 [========>.....................] - ETA: 3:48 - loss: 9.6150

KeyboardInterrupt: 

In [ ]:
model.fit(train_set, validation_data=validation_set, epochs=2)

2021-12-08 00:32:49.622711: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


Epoch 1/2


2021-12-08 00:32:55.524026: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


24/75 [========>.....................] - ETA: 3:48 - loss: 9.6150

KeyboardInterrupt: 

In [ ]:
model.fit(train_set, validation_data=validation_set, epochs=2)

2021-12-08 00:32:49.622711: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


Epoch 1/2


2021-12-08 00:32:55.524026: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


24/75 [========>.....................] - ETA: 3:48 - loss: 9.6150

KeyboardInterrupt: 

In [11]:
configuration = BertConfig(
        vocab_size=tokenizer.vocab_size,
        hidden_size=768,
        num_hidden_layers=12,
        num_attention_heads=12,
        intermediate_size=3072,
        hidden_act="gelu",
        hidden_dropout_prob=0.1,
        attention_probs_dropout_prob=0.1,
        max_position_embeddings=512,
        type_vocab_size=2,
        initializer_range=0.02,
        layer_norm_eps=1e-12,
        pad_token_id=0,
        position_embedding_type="absolute",
        use_cache=True,
        classifier_dropout=None,
)

model = TFBertForMaskedLM(
    configuration
)

learning_rate = 5e-5 #2e-5
weight_decay = 0.01

optimizer = AdamWeightDecay(learning_rate=learning_rate, weight_decay_rate=weight_decay)

model.compile(
    optimizer = optimizer
    # optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5),
    # loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    # metrics=tf.metrics.SparseCategoricalAccuracy(),
)

2021-12-08 00:39:57.691044: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2021-12-08 00:39:57.691174: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! Please ensure your labels are passed as the 'labels' key of the input dict so that they are accessible to the model during the forward pass. To disable this behaviour, please pass a loss argument, or explicitly pass loss=None if you do not want your model to compute a loss.


Metal device set to: Apple M1

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



In [12]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15,
return_tensors="tf"
)

In [14]:
train_set = tokenized_datasets["train"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "labels"],
    shuffle=True,
    batch_size=64,
    collate_fn=data_collator,
)

validation_set = tokenized_datasets["dev"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "labels"],
    shuffle=False,
    batch_size=64,
    collate_fn=data_collator,
)

In [15]:
model.fit(train_set, validation_data=validation_set, epochs=2)

2021-12-08 00:41:44.606636: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


Epoch 1/2


2021-12-08 00:41:50.768750: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


In [22]:
tokenizer(dataset[1000]["text"])

{'input_ids': [2, 58, 1937, 4453, 16, 3024, 1091, 4372, 54, 29, 5371, 106, 5729, 39, 4869, 123, 13786, 5, 656, 423, 31, 2771, 485, 5, 281, 1], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [34]:
sent = dataset[1000]["text"]
n = 4
sent_mask = " ".join([t if i!=n else "[MASK]" for i,t in enumerate(sent.split())])
sent_mask

'then insteadof struggl ing [MASK] ain toprevent them from reaching my consciousness i stepped back asitwere and let thought sand feelings come and go'

In [35]:
outputs = model(tokenizer(sent_mask, return_tensors="tf")["input_ids"])
outputs

TFMaskedLMOutput(loss=None, logits=<tf.Tensor: shape=(1, 26, 26349), dtype=float32, numpy=
array([[[-4.4881043 , -1.1281193 ,  1.2651832 , ..., -3.865802  ,
         -3.8548353 , -4.341054  ],
        [-4.8321047 , -0.9831344 , -0.8014428 , ..., -4.4701204 ,
         -3.5916226 , -4.391395  ],
        [-4.200804  , -0.7827609 , -1.0982523 , ..., -4.4545183 ,
         -4.2676077 , -4.4787583 ],
        ...,
        [-4.3038697 , -0.9855108 , -1.7016997 , ..., -4.840664  ,
         -4.049444  , -4.031029  ],
        [-4.483965  , -1.14926   , -0.79288185, ..., -3.6548307 ,
         -4.109019  , -3.921715  ],
        [-4.1470866 ,  2.1915812 , -0.46791494, ..., -3.9917128 ,
         -3.7296255 , -3.6430888 ]]], dtype=float32)>, hidden_states=None, attentions=None)

In [86]:
sent

'then insteadof struggl ing inv ain toprevent them from reaching my consciousness i stepped back asitwere and let thought sand feelings come and go'

In [95]:
parad = [[(tokenizer.ids_to_tokens[k],v) for v,k in sorted([(v,i) for i,v in enumerate(outputs.logits[:,p].numpy().tolist()[0])], reverse=True)] for p in range(20)]

In [96]:
[parad[i][0] for i in range(len(parad))]

[('ing', 4.185389995574951),
 ('then', 6.417963027954102),
 ('the', 3.9221103191375732),
 ('s', 2.9763553142547607),
 ('ing', 9.62474250793457),
 ('and', 4.000395774841309),
 ('the', 3.569209575653076),
 ('their', 4.091992378234863),
 ('them', 8.237309455871582),
 ('from', 7.786880016326904),
 ('in', 2.7499029636383057),
 ('my', 6.577600002288818),
 ('and', 3.457987070083618),
 ('i', 7.709474563598633),
 ('to', 3.0285086631774902),
 ('back', 5.972971439361572),
 ('and', 3.247152805328369),
 ('and', 11.189146995544434),
 ('in', 4.0166015625),
 ('thought', 4.256227016448975)]

In [98]:
parad[-1]

[('thought', 4.256227016448975),
 ('inthe', 3.3842475414276123),
 ('s', 3.319896936416626),
 ('the', 3.2724437713623047),
 ('at', 3.0720911026000977),
 ('with', 2.986698865890503),
 ('to', 2.9801266193389893),
 ('this', 2.7567028999328613),
 ('ing', 2.732544183731079),
 ('are', 2.6277990341186523),
 ('a', 2.617731809616089),
 ('as', 2.606462001800537),
 ('that', 2.55509614944458),
 ('is', 2.499570369720459),
 ('then', 2.4564075469970703),
 ('of', 2.4553709030151367),
 ('it', 2.4082751274108887),
 ('but', 2.3524272441864014),
 ('ed', 2.3307981491088867),
 ('one', 2.2417314052581787),
 ('my', 2.1435656547546387),
 ('in', 2.107780933380127),
 ('ly', 2.093127489089966),
 ('or', 2.071748733520508),
 ('itis', 2.007465362548828),
 ('ofthe', 1.9947885274887085),
 ('c', 1.9852861166000366),
 ('was', 1.9839030504226685),
 ('i', 1.9651674032211304),
 ('which', 1.9509996175765991),
 ('and', 1.943827748298645),
 ('art', 1.9253251552581787),
 ('ate', 1.9010833501815796),
 ('not', 1.869430422782898),